In [1]:
import sys
 
sys.path.insert(0, '..')

In [2]:
from summarizer.topic_sum import create_topic_summarizer

In [3]:
import pandas as pd
import numpy as np
import random
import logging


random.seed(93)
np.random.seed(93)

# The code expects a dataframe with title, published, topic, and probability columns

topic_df = pd.read_parquet("gs://scraped-news-article-data-null/2023-topics.parquet")
topic_df.head()

,source,id,category,title,published,body,summary,summary_type,topic,probability
0,reuters,29667,Europe,European shares rise on upbeat cues from Powel...,2023-02-08T17:11:00,Feb 8 (Reuters) - European shares rose on Wedn...,* \n* STOXX 600 pulls back from 9-mth high to ...,BULLETS,-1,0.000000
1,reuters,79619,China,China Evergrande debt restructuring incentive ...,2023-04-27T06:05:00,"HONG KONG, April 27 (Reuters) - Embattled prop...",,NULL,310,0.281727
2,reuters,48010,European Markets,European stocks rally as banking worries fade,2023-03-30T16:27:00,March 30 (Reuters) - European stocks rose to n...,* \n* H&M posts surprise profit in Dec-Feb\n* ...,BULLETS,77,1.000000
3,reuters,67859,Aerospace & Defense,China gears up to compete with SpaceX's Starli...,2023-03-02T10:29:00,"BEIJING, March 2 (Reuters) - China's military-...",,NULL,-1,0.000000
4,reuters,109054,World,Russia's war on Ukraine latest: Moscow denies ...,2023-05-12T05:13:00,May 11 (Reuters) - Russia's defence ministry o...,,NULL,719,1.000000


In [4]:
api_summarizer = create_topic_summarizer(temperature=0)
#test = api_summarizer(topic_df, 0)
#print(test)

In [5]:
with open("/home/jupyter/apikey", "r") as api_fp:
    api_key = api_fp.read().strip()

openai_summarizer = create_topic_summarizer(kind="openai", api_key=api_key, temperature=0)
test = openai_summarizer(topic_df, 0)
print(test)

The articles discuss various developments and applications of artificial intelligence (AI) technology, particularly in the form of generative language models like ChatGPT and Bard. Companies like Google, Microsoft, and OpenAI are investing heavily in AI and developing new products and features that incorporate the technology. AI is being used in a variety of industries, including finance, real estate, and healthcare, and is seen as a potential tool for improving productivity and employee engagement. However, there are also concerns about the risks and ethical implications of AI, and regulators are beginning to investigate the technology.


In [6]:
topics = list(set(topic_df.topic.unique()) - {-1})
print(len(topics))

1564


In [7]:
from tqdm import tqdm


tqdm.pandas()

In [8]:
summarizer_type = "openai"


try:
    topic_sum = pd.read_parquet("gs://scraped-news-article-data-null/2023-topics-%s.parquet" % summarizer_type)
except:
    topic_sum = pd.DataFrame({
        "topics": topics,
        "summary": ["" for _ in topics]
    })

topic_sum.head()

,topics,summary
0,0,The news articles revolve around the advanceme...
1,1,The news articles cover the fluctuations in oi...
2,2,The news articles revolve around the security ...
3,3,The Federal Reserve's interest rate decisions ...
4,4,The news articles cover various aspects of the...


In [ ]:
works = set(topic_sum.loc[topic_sum.summary.str.len() == 0]["topics"].to_list())
summarizer = create_topic_summarizer(summarizer_type, api_key=api_key)
with tqdm(total = len(works)) as progress:
    for i, w in enumerate(works):
        topic = topic_sum.loc[topic_sum.topics == w].iloc[0]["topics"]
        summary = summarizer(topic_df, topic)
        topic_sum.loc[topic_sum.topics == w, "summary"] = summary
        progress.update(1)
        if i % 10 == 0:
            topic_sum.to_parquet("gs://scraped-news-article-data-null/2023-topics-%s.parquet" % summarizer_type, index=False)
topic_sum.to_parquet("gs://scraped-news-article-data-null/2023-topics-%s.parquet" % summarizer_type, index=False)

  5%|▍         | 57/1247 [16:41<6:15:13, 18.92s/it]